<a href="https://colab.research.google.com/github/Lonely52Hz/ID2223_Scalable_Machine_Learning_and_Deep_Learning/blob/main/ID2223_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set environment in colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
!pip install pyspark

     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 198 kB 49.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=50e584181db8b2535232175ccd8feedf1999162d96350c7b9757e56d7ad9246d
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port", "4050").getOrCreate()
from pyspark.sql.functions import *
#from pyspark.sql.types import *

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [5]:
# read files
train = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
test = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-test.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
dev = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-dev.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')

train.show(5)
train.printSchema()

+-------------+------+--------+-----+-----+--------------------+--------------------+
|        genre|  file|    year|index|score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+-----+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|  5.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|  3.8|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|  3.8|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|  2.6|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9| 4.25|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+-----+--------------------+--------------------+
only showing top 5 rows

root
 |-- genre: string (nullable = true)
 |-- file: string (nullable = true)
 |-- year: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- sentenc

In [6]:
# map [0, 5] to [-1, 1]
train = train.withColumn('score', col('score')/2.5 - 1)
test = test.withColumn('score', col('score')/2.5 - 1)
dev = dev.withColumn('score', col('score')/2.5 - 1)

train.show(5)
train.describe(['score']).show()

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|                 0.7|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

+-------+-------------------+
|summary|              score|
+-----

In [7]:
# find rows with missing value
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()
dev.select([count(when(col(c).isNull(), c)).alias(c) for c in dev.columns]).show()

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        6|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+



In [8]:
# delete rows with missing value
train = train.na.drop()
test = test.na.drop()
dev = dev.na.drop()

train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
train.show(5)

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        0|
+-----+----+----+-----+-----+---------+---------+

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|     

In [9]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models, losses, InputExample

     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 28.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.9 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 61 kB 515 kB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
     |████████████████████████████████| 596 kB 57.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=02cfdb78d5f56c4a59cd23f4e62b3bffd92e5d492315150f830b0d524f121729
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),\
                               pooling_mode_mean_tokens=True,\
                               pooling_mode_cls_token=False,\
                               pooling_mode_max_tokens=False)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [12]:
loss = losses.CosineSimilarityLoss(model)

In [19]:
from torch.utils.data import DataLoader

example = []
sa=[]
sb=[]
scor=[]
df_train=train.toPandas()
# print(train)
# print(df_train)
for row in df_train.iloc:
  row=row.values.tolist()
  #print(row)
  
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  sa.append(row[5])
  sb.append(row[6])
  scor.append(row[4])
  example.append(inp_example)
train_dataloader = DataLoader(example, shuffle=True, batch_size=16)

In [ ]:
from sentence_transformers import evaluation
evaluator = evaluation.EmbeddingSimilarityEvaluator(sa, sb, scor)

model.fit(train_objectives=[(train_dataloader, loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/359 [00:00<?, ?it/s]

In [25]:
# evaluation
from scipy import stats

In [32]:
from sentence_transformers import util
a=model.encode('A girl is styling her hair.')
b=model.encode('A girl is brushing her hair.')
cos_sim = util.pytorch_cos_sim(a, b)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.3124]])
